# Notebook #2: Carga: creación de Base de Datos SQL

En este segundo notebook crearemos y cargaremos nuestra base de datos.

- El primer paso será importar las librerías necesarias:

In [9]:
# Librerías para tratamiento de datos

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones.
from src import soporte_variables as sv

# Librería para trabajar con bases de datos SQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

- Ahora importaremos los CSVs creados en el notebook anterior, y crearemos listas de tuplas, que es el formato necesario para la carga a la base de datos.
- Para ello utilizaremos la función `sf.csvs_a_tuplas()`, que recibe como parámetros una lista con los directorios de los CSVs que queremos transformar, y devuelve una lista de tuplas para cada uno, que podemos asignar directamente a las variables.
- Los geojson los trataremos más adelante, dado que requieren un tratamiento especial.

In [3]:
rutas = ["../datos/finales/ingresos_hogares.csv",
"../datos/finales/poblacion.csv",
"../datos/finales/redpiso.csv"]

lista_ingresos, lista_poblacion, lista_redpiso = sf.csvs_a_tuplas(rutas)

- El tercer paso consiste en crear la base de datos, para lo cual, usaremos la función `dbeaver_crear_db()`, que recibe como único argumento el nombre de la base de datos que deseamos crear. En su código, se crea la conexión a DBeaver, utilizando los parámetros de conexión (usuario y contraseña) que se han guardado en el soporte `../src/.env` (oculto) e importado en  `../src/soporte_funciones.py`.

- Llamaremos a la base de datos "AlquileresMadrid".

In [ ]:
#sf.dbeaver_crear_db("alquileresmadrid")

Base de datos AlquileresMadrid creada con éxito


- Los datos se insertarán en las tablas que crearemos a continuación, con ayuda de las queries de creación definidas en `../src/soporte_variables.py`.

- Para crear esas tablas, hemos decidido que nuestra base de datos se estructurará de la manera en que vemos en el siguiente esquema entidad relación, que muestra la información que contiene cada tabla y cómo se relaciona entre sí. En el README del proyecto se explica su diseño.

<img src="../images/DiagramaER.png" width="400">


- En este caso utilizaremos dos funciones:

    - `sf.dbeaver_conexion()`: recibe como parámetro el nombre de la base de datos de DBeaver y crea la conexión entre el notebook y la base de datos.
    - `sf.dbeaver_commit()`: recibe como parámetros la conexión a DBeaver y la query de creación, realizando el commit hacia la base de datos.

In [2]:
sf.dbeaver_commit(sf.dbeaver_conexion("alquileresmadrid"),sv.query_creacion_distritos)
sf.dbeaver_commit(sf.dbeaver_conexion("alquileresmadrid"),sv.query_creacion_airbnb)
sf.dbeaver_commit(sf.dbeaver_conexion("alquileresmadrid"),sv.query_creacion_idealista)
sf.dbeaver_commit(sf.dbeaver_conexion("alquileresmadrid"),sv.query_creacion_redpiso)
sf.dbeaver_commit(sf.dbeaver_conexion("alquileresmadrid"),sv.query_creacion_ingreso_hogar)
sf.dbeaver_commit(sf.dbeaver_conexion("alquileresmadrid"),sv.query_creacion_poblacion)

Commit realizado
Commit realizado
Commit realizado
Commit realizado
Commit realizado
Commit realizado


- Con las tablas preparadas, continuamos con la inserción de los mismos. Primero insertaremos manualmente las tablas que contienen información geoespacial, por necesitar un tratamiento especial, y luego seguiremos con los csv tradicionales.
- Empezaremos por los importar los archivos de tipo `geojson` que se han creado en el notebook #1.

In [8]:
gdf_distritos = gpd.read_file("../datos/finales/distritos.geojson")
gdf_airbnb = gpd.read_file("../datos/finales/airbnb.geojson")
gdf_idealista = gpd.read_file("../datos/finales/idealista.geojson")

- Por orden de inserción, debemos empezar por las tablas que sólo contienen Primary Keys, en este caso, distritos.
- Usaremos la función de conexión a DBeaver `sf.dbeaver_conexion()`, que recibe como argumento el nombre de la base de datos, así como las queries de inserción que hemos definido en `src/soporte_variables.py`.
- Debido a que cada DF tiene un formato distinto, no tiene sentido automatizar este proceso en una función.
- Además de nuestras tuplas habituales para cada columna, en este caso, para las columnas que contienen datos geogmétricos, debemos exportarlas como `.wkt`. El formato WKT (Well Known Text), es un formato de codificación específicamente diseñado para la caracterización y almacenamiento de objetos geométricos espaciales en formato vectorial.

In [ ]:
# Conexión a base de datos y cursor
conn = sf.dbeaver_conexion("alquileresmadrid")
cur = conn.cursor()

# Insertamos datos de cada GeoDataFrame:
for _, fila in gdf_distritos.iterrows(): # Con la barra baja (_) indicamos que no queremos utilizar el índice del DF.
    id_distrito = fila['ID_Distrito']
    nombre = fila['Distrito']
    geom = fila['geometry'].wkt

    cur.execute(sv.query_inser_distritos, (id_distrito, nombre, geom))

# Commit y cerrar
conn.commit()
cur.close()
conn.close()

- Para evitar errores cerramos el cursor y la conexión tras cada inserción. Seguimos con la tabla de AirBnB.

In [ ]:
# Conexión a base de datos y cursor
conn = sf.dbeaver_conexion("alquileresmadrid")
cur = conn.cursor()

# Insertamos datos de cada GeoDataFrame:
for _, fila in gdf_airbnb.iterrows():
    id_distrito2 = fila['ID_Distrito']
    precio2 = fila['Precio Total']
    descripcion2 = fila['Descripcion']
    geom2 = fila['geometry'].wkt
    
    cur.execute(sv.query_inser_airbnb, (id_distrito2, precio2, descripcion2, geom2))

# Commit y cerrar
conn.commit()
cur.close()
conn.close()

Y finalmente la tabla de Idealista:

In [10]:
# Conexión a base de datos y cursor
conn = sf.dbeaver_conexion("alquileresmadrid")
cur = conn.cursor()

# Insertamos datos de cada GeoDataFrame:
for _, fila in gdf_idealista.iterrows():
    id_distrito3 = fila['ID_Distrito']
    precio3 = fila['Precio']
    tipo3 = fila['Tipo']
    planta3 = fila['Planta']
    tamanio3 = fila['Tamanio']
    habitaciones3 = fila['Habitaciones']
    banios3 = fila['Banios']
    direccion3 = fila['Direccion']
    descripcion3 = fila['Descripcion']
    geom3 = fila['geometry'].wkt
    
    cur.execute(sv.query_inser_idealista, (id_distrito3, precio3, tipo3, planta3, tamanio3, habitaciones3, banios3, direccion3, descripcion3, geom3))

# Commit y cerrar
conn.commit()
cur.close()
conn.close()

- Insertados los datos en las tablas que contienen datos de geolocalización, continuamos con las tres restantes: ingresos por hogar, población y Redpiso.

- Usaremos la función `sf.dbeaver_commitmany()`, que recibe como argumentos:
    - la conexión a DBeaver (usando la función de conexión),
    - las queries de inserción que hemos definido en `src/soporte_variables.py` y,
    - los datos que deseamos insertar, en este caso, las listas de tuplas preparadas anteriormente.
- Esta función envía múltiples datos desde el notebook hacia la base de datos DBeaver.

In [25]:
sf.dbeaver_commitmany(sf.dbeaver_conexion("alquileresmadrid"),sv.query_inser_ingreso_hogar,lista_ingresos)

Commit realizado


In [19]:
sf.dbeaver_commitmany(sf.dbeaver_conexion("alquileresmadrid"),sv.query_inser_poblacion,lista_poblacion)

Commit realizado


In [20]:
sf.dbeaver_commitmany(sf.dbeaver_conexion("alquileresmadrid"),sv.query_inser_redpiso,lista_redpiso)

Commit realizado


- Con los datos insertados, podemos empezar a trabajar sobre nuestra base de datos. Continuamos con las consultas a la base de datos, la visualización y el análisis en el Notebook #3.